In [ ]:
from osgeo import gdal
import os

# Caminho dos arquivos
input_file = '/home/helvecioneto/goesgcp/tmp/OR_ABI-L2-CMIPF-M6C02_G16_s20241531500188_e20241531509496_c20241531509557.nc'
output_file = '/home/helvecioneto/goesgcp/teste.nc'
base = "/home/helvecioneto/goesgcp/grade_1km.nc"  # Arquivo NetCDF que será usado como cutline

# Verifique se o arquivo de entrada existe
if not os.path.exists(input_file):
    print(f"Arquivo de entrada não encontrado: {input_file}")
    exit()

# Verifique se o arquivo de base existe
if not os.path.exists(base):
    print(f"Arquivo de referência não encontrado: {base}")
    exit()

# Abra o arquivo de entrada
src_ds = gdal.Open(input_file)
if src_ds is None:
    print(f"Erro ao abrir o arquivo de entrada: {input_file}")
    exit()

# Abra o arquivo de referência (base) que será usado como cutline
ref_ds = gdal.Open(base)
if ref_ds is None:
    print(f"Erro ao abrir o arquivo de referência: {base}")
    exit()

# Obtenha as informações de resolução do arquivo de referência
geo_transform = ref_ds.GetGeoTransform()
res_x = geo_transform[1]  # Resolução X
res_y = geo_transform[5]  # Resolução Y (geralmente negativa)

# Verifique se as resoluções estão corretas
print(f"Resolução X: {res_x}, Resolução Y: {res_y}")

# Configurar o dicionário de parâmetros kwargs
kwargs = {
    'cutlineDSName': base,  # Arquivo NetCDF de recorte
    'cropToCutline': True,  # Recortar para a área definida no cutline
    'xRes': res_x,  # Resolução X
    'yRes': res_y,  # Resolução Y
    'resampleAlg': gdal.GRA_Bilinear,  # Algoritmo de interpolação (bilinear)
    'format': 'NetCDF'  # Garantir o formato de saída
}

# Tente realizar o recorte
try:
    gdal.Warp(output_file, src_ds, **kwargs)
    print(f"Arquivo gerado com sucesso: {os.path.abspath(output_file)}")
except Exception as e:
    print(f"Ocorreu um erro ao executar gdal.Warp: {e}")

# Feche os datasets
src_ds = None
ref_ds = None


Resolução X: 1.0, Resolução Y: 1.0
Arquivo gerado com sucesso: /home/helvecioneto/goesgcp/teste.nc


ERROR 1: Cannot open grade_1km.nc.


In [3]:
!ls tmp

20240601_1500.nc_tmp1.nc
OR_ABI-L2-CMIPF-M6C02_G16_s20241531500188_e20241531509496_c20241531509557.nc


In [5]:
from osgeo import gdal, osr

# Caminho dos arquivos
input_file = 'tmp/OR_ABI-L2-CMIPF-M6C02_G16_s20241531500188_e20241531509496_c20241531509557.nc'
output_file = 'teste.nc'
var_name = 'CMI'

lon_min = -60.0
lat_min = -40.0
lon_max = -30.0
lat_max = -10.0
resolution = 0.01

# Abra o arquivo de entrada CMI
src_ds = gdal.Open(f"netCDF:{input_file}:{var_name}")

# Get source projection
source_prj = osr.SpatialReference()
source_prj.ImportFromProj4(src_ds.GetProjectionRef())

# Set target projection
target_prj = osr.SpatialReference()
target_prj.ImportFromProj4("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

print("Source Projection:", source_prj.ExportToWkt())
print("Target Projection:", target_prj.ExportToWkt())


# Configurar o dicionário de parâmetros kwargs
kwargs = {
        'format': 'netCDF',
        'srcSRS': source_prj.ExportToWkt(),
        'dstSRS': target_prj.ExportToWkt(),
        'outputBounds': (lon_min, lat_min, lon_max, lat_max),
        'xRes': resolution,
        'yRes': resolution,
        'resampleAlg': gdal.GRA_Bilinear,
        'outputType': gdal.GDT_Int16,
        'dstNodata': -9999,
    }


# Tente realizar o recorte
gdal.Warp(output_file, src_ds, **kwargs)

# Feche os datasets
src_ds = None

ERROR 4: netCDF:tmp/OR_ABI-L2-CMIPF-M6C02_G16_s20241531500188_e20241531509496_c20241531509557.nc:CMI: No such file or directory


AttributeError: 'NoneType' object has no attribute 'GetProjectionRef'